In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import os

# Make sure the output directory exists
# Assuming you run this from a script in a 'scripts' folder, and 'data' and 'assets' are siblings
os.makedirs('../assets', exist_ok=True)
data_file_path = '../data/bubble_plot.csv' # Ensure this path is correct

# Load the data
try:
    df = pd.read_csv(data_file_path)
except FileNotFoundError:
    print(f"Error: The data file was not found at {data_file_path}")
    print("Please ensure the path to 'bubble_plot.csv' is correct.")
    exit()

# Clean the data
df['GDP'] = pd.to_numeric(df['GDP, PPP (current international $) [NY.GDP.MKTP.PP.CD]'].str.replace(',', ''), errors='coerce')
df['Population'] = pd.to_numeric(df['Population, total [SP.POP.TOTL]'], errors='coerce')
df['Departures'] = pd.to_numeric(df['International tourism, number of departures [ST.INT.DPRT]'].str.replace(',', ''), errors='coerce')
df['PerCapita'] = pd.to_numeric(df['International Tourism Departures per capita'], errors='coerce')

# Calculate GDP per capita (new for x-axis)
df['GDP_Per_Capita'] = df['GDP'] / df['Population']

# Create continent mapping
continent_mapping = {
    'Denmark': 'Europe', 'Sweden': 'Europe', 'Norway': 'Europe', 'Finland': 'Europe', 'Iceland': 'Europe',
    'United Kingdom': 'Europe', 'Germany': 'Europe', 'France': 'Europe', 'Italy': 'Europe', 'Spain': 'Europe',
    'Netherlands': 'Europe', 'Belgium': 'Europe', 'Switzerland': 'Europe', 'Austria': 'Europe', 'Poland': 'Europe',
    'Czech Republic': 'Europe', 'Hungary': 'Europe', 'Romania': 'Europe', 'Portugal': 'Europe', 'Greece': 'Europe',
    'Ireland': 'Europe', 'Luxembourg': 'Europe', 'Slovenia': 'Europe', 'Croatia': 'Europe', 'Lithuania': 'Europe',
    'Latvia': 'Europe', 'Estonia': 'Europe', 'Slovakia': 'Europe', 'Bulgaria': 'Europe',
    'United States': 'North America', 'Canada': 'North America', 'Mexico': 'North America',
    'China': 'Asia', 'Japan': 'Asia', 'Korea, Rep.': 'Asia', 'India': 'Asia', 'Indonesia': 'Asia',
    'Singapore': 'Asia', 'Malaysia': 'Asia', 'Thailand': 'Asia', 'Vietnam': 'Asia', 'Philippines': 'Asia',
    'Hong Kong SAR, China': 'Asia', 'Russian Federation': 'Asia', 'Taiwan, China': 'Asia',
    'Australia': 'Oceania', 'New Zealand': 'Oceania',
    'South Africa': 'Africa', 'Egypt, Arab Rep.': 'Africa', 'Nigeria': 'Africa', 'Kenya': 'Africa', 'Morocco': 'Africa',
    'Brazil': 'South America', 'Argentina': 'South America', 'Chile': 'South America', 'Colombia': 'South America',
    'Peru': 'South America', 'Venezuela, RB': 'South America',
    'Saudi Arabia': 'Middle East', 'United Arab Emirates': 'Middle East', 'Turkey': 'Middle East', 'Israel': 'Middle East',
    'Albania': 'Europe', 'Belarus': 'Europe', 'Bosnia and Herzegovina': 'Europe', 'Kazakhstan': 'Asia',
    'Mongolia': 'Asia', 'North Korea': 'Asia', 'Sri Lanka': 'Asia', 'Pakistan': 'Asia', 'Bangladesh': 'Asia',
    'Myanmar': 'Asia', 'Nepal': 'Asia', 'Lebanon': 'Middle East', 'Jordan': 'Middle East', 'Kuwait': 'Middle East',
    'Bahrain': 'Middle East', 'Qatar': 'Middle East', 'Oman': 'Middle East', 'Yemen': 'Middle East',
    'Ethiopia': 'Africa', 'Ghana': 'Africa', 'Tanzania': 'Africa', 'Uganda': 'Africa', 'Zambia': 'Africa',
    'Zimbabwe': 'Africa', 'Cameroon': 'Africa', 'Senegal': 'Africa', 'Mali': 'Africa', 'Niger': 'Africa',
    'Chad': 'Africa', 'Sudan': 'Africa', 'Libya': 'Africa',
    'Tunisia': 'Africa', 'Algeria': 'Africa',
    'Ecuador': 'South America', 'Bolivia': 'South America', 'Paraguay': 'South America', 'Uruguay': 'South America',
    'Papua New Guinea': 'Oceania', 'Fiji': 'Oceania', 'Solomon Islands': 'Oceania'
}

df['Continent'] = df['Country Name'].map(continent_mapping).fillna('Other')
df_clean = df.dropna(subset=['GDP_Per_Capita', 'PerCapita', 'Population', 'Continent']).copy()
years = sorted(df_clean['Time'].unique())
years = [year for year in years if 1996 <= year <= 2019]

# Modern, more vibrant color scheme
continent_colors = {
    'Europe': '#3498db',         # Bright blue
    'North America': '#9b59b6',  # Purple
    'Asia': '#2ecc71',           # Green
    'Oceania': '#e74c3c',        # Red
    'Africa': '#f39c12',         # Orange
    'South America': '#1abc9c',  # Turquoise
    'Middle East': '#34495e'     # Dark blue-gray
}

df_clean = df_clean[df_clean['Continent'] != 'Other'].copy()

if df_clean.empty or not years:
    print("Dataframe is empty after cleaning or no valid years found. Cannot proceed.")
    exit()

df_clean['IsDenmark'] = df_clean['Country Name'] == 'Denmark'

# Create a function to calculate bubble sizes - SMALLER than before
def calculate_bubble_size(population):
    # Reduced scaling factor for smaller bubbles
    return np.sqrt(population / 5000000) * 6  # Reduced from 10 to 6, and increased divisor

fig = go.Figure()
current_trace_idx = 0
# Add legend traces
for continent in continent_colors.keys():
    fig.add_trace(go.Scatter(x=[None], y=[None], mode='markers', name=continent,
                             marker=dict(size=8, color=continent_colors[continent]),
                             showlegend=True, legendgroup=continent))
    current_trace_idx += 1
fig.add_trace(go.Scatter(x=[None], y=[None], mode='markers', name='🇩🇰 Denmark',
                         marker=dict(size=12, color='#003f5c', line=dict(width=2, color='#C60C30'), symbol='star'),
                         showlegend=True, legendgroup='Denmark'))
current_trace_idx += 1

data_trace_indices = []
continent_order_for_traces = []
first_year = years[0]

# Add initial data traces for the first year
for continent in continent_colors.keys():
    data_trace_indices.append(current_trace_idx)
    continent_order_for_traces.append(continent)
    continent_data_initial = df_clean[(df_clean['Time'] == first_year) & (df_clean['Continent'] == continent) & (~df_clean['IsDenmark'])]
    fig.add_trace(go.Scatter(
        x=continent_data_initial['GDP_Per_Capita'], y=continent_data_initial['PerCapita'],
        mode='markers', name=continent, text=continent_data_initial['Country Name'],
        hovertemplate='<b>%{text}</b><br>GDP per Capita: $%{x:,.0f}<br>Tourism/Capita: %{y:.2f}<br><extra></extra>',
        marker=dict(size=calculate_bubble_size(continent_data_initial['Population']) if not continent_data_initial.empty else [],
                    sizemin=4, color=continent_colors[continent], opacity=0.8, line=dict(width=0.5, color='white')),
        visible=True, legendgroup=continent, showlegend=False))
    current_trace_idx += 1

data_trace_indices.append(current_trace_idx)
continent_order_for_traces.append('Denmark')
denmark_data_initial = df_clean[(df_clean['Time'] == first_year) & (df_clean['IsDenmark'])]
fig.add_trace(go.Scatter(
    x=denmark_data_initial['GDP_Per_Capita'], y=denmark_data_initial['PerCapita'],
    mode='markers', name='Denmark', text=(['Denmark'] * len(denmark_data_initial)) if not denmark_data_initial.empty else [],
    hovertemplate='<b>Denmark</b><br>GDP per Capita: $%{x:,.0f}<br>Tourism/Capita: %{y:.2f}<br><extra></extra>',
    marker=dict(size=calculate_bubble_size(denmark_data_initial['Population']) if not denmark_data_initial.empty else [],
                color='#003f5c', line=dict(width=2, color='#C60C30'), symbol='star'),
    visible=True, legendgroup='Denmark', showlegend=False))

# --- Create Frames for Animation ---
plotly_frames = []
for year in years:
    frame_data_payload = []
    for continent_or_denmark in continent_order_for_traces:
        if continent_or_denmark == 'Denmark':
            current_year_data = df_clean[(df_clean['Time'] == year) & (df_clean['IsDenmark'])]
            marker_size = calculate_bubble_size(current_year_data['Population']).tolist() if not current_year_data.empty else []
            trace_update_dict = {'x': current_year_data['GDP_Per_Capita'].tolist(),
                                 'y': current_year_data['PerCapita'].tolist(),
                                 'marker': {'size': marker_size},
                                 'text': (['Denmark'] * len(current_year_data)) if not current_year_data.empty else []}
        else:
            current_year_data = df_clean[(df_clean['Time'] == year) & (df_clean['Continent'] == continent_or_denmark) & (~df_clean['IsDenmark'])]
            marker_size = calculate_bubble_size(current_year_data['Population']).tolist() if not current_year_data.empty else []
            trace_update_dict = {'x': current_year_data['GDP_Per_Capita'].tolist(),
                                 'y': current_year_data['PerCapita'].tolist(),
                                 'marker': {'size': marker_size},
                                 'text': current_year_data['Country Name'].tolist()}
        frame_data_payload.append(trace_update_dict)
    plotly_frames.append(go.Frame(data=frame_data_payload, name=str(year), traces=data_trace_indices))
fig.frames = plotly_frames

# Create slider steps
steps = []
for year in years:
    step = dict(
        method="animate",
        args=[[str(year)],
              {"frame": {"duration": 0, "redraw": True},
               "mode": "immediate",
               "transition": {"duration": 0}}
             ],
        label=str(year)
    )
    steps.append(step)

# Improved layout with GDP per capita on x-axis
fig.update_layout(
    title="Denmark's Tourism Position in Global Context (1996-2019)",
    xaxis=dict(title="GDP per Capita (USD, PPP)", type='log',
               range=[np.log10(max(0.01, df_clean['GDP_Per_Capita'].min(skipna=True) * 0.8 if pd.notna(df_clean['GDP_Per_Capita'].min(skipna=True)) else 0.01)),
                      np.log10(max(1, df_clean['GDP_Per_Capita'].max(skipna=True) * 1.2 if pd.notna(df_clean['GDP_Per_Capita'].max(skipna=True)) else 1))],
               tickmode='array',
               tickvals=[500, 1000, 2000, 5000, 10000, 20000, 50000, 100000],
               ticktext=['$500', '$1K', '$2K', '$5K', '$10K', '$20K', '$50K', '$100K'],
               showgrid=True, gridcolor='#eeeeee'),
    yaxis=dict(title="International Tourism Departures per Capita",
               showgrid=True, 
               gridcolor='#eeeeee',
               range=[-0.2, 2.5],  # Reduced range to focus on most data points
               fixedrange=True),   # This will lock the range to prevent auto-adjustment
    showlegend=True,
    legend=dict(orientation="v", yanchor="top", y=0.99, xanchor="left", x=1.02, 
                bgcolor="rgba(255,255,255,0.8)", bordercolor="lightgray", borderwidth=1),
    plot_bgcolor='white', width=None, height=600, autosize=True,
    margin=dict(l=60, r=100, t=80, b=120),
    annotations=[
        dict(
            x=0.99, y=0.01,
            xref="paper", yref="paper",
            text="Note: Bubble size represents population",
            showarrow=False,
            font=dict(size=10, color="gray")
        )
    ],
    updatemenus=[{
        'type': 'buttons', 'showactive': False, 'y': -0.3, 'x': 0.5, 'xanchor': 'center',
        'yanchor': 'top', 'direction': 'left', 'pad': {'t': 10,'b':10, 'r': 10},
        'buttons': [
            {'label': '▶️ Play', 'method': 'animate',
             'args': [None,
                      {'frame': {'duration': 800, 'redraw': True},
                       'fromcurrent': True,
                       'transition': {'duration': 300, 'easing': 'quadratic-in-out'},
                       'mode': 'immediate',
                       'autoplay': False}]},
            {'label': '⏸️ Pause', 'method': 'animate',
             'args': [[None],
                      {'frame': {'duration': 0, 'redraw': False},
                       'mode': 'immediate',
                       'transition': {'duration': 0}}]}
        ]
    }],
    sliders=[{
        'active': 0,
        'currentvalue': {'prefix': 'Year: ', 'visible': True, 'xanchor': 'right', 
                         'font': {'size': 14, 'color': '#333333'}},
        'pad': {'t': 25, 'b': 45},
        'steps': steps,
        'len': 0.9,
        'x': 0.05,
        'y': 0
    }]
)

# Add a line trace showing the global median trend of GDP per Capita vs Tourism per Capita
# This helps provide context for Denmark's position
median_by_year = df_clean.groupby('Time')[['GDP_Per_Capita', 'PerCapita']].median().reset_index()
fig.add_trace(go.Scatter(
    x=median_by_year['GDP_Per_Capita'],
    y=median_by_year['PerCapita'],
    mode='lines',
    line=dict(color='rgba(0,0,0,0.3)', width=1, dash='dot'),
    name='Global Median Trend',
    hovertemplate='Global Median<br>Year: %{text}<br>GDP per Capita: $%{x:,.0f}<br>Tourism/Capita: %{y:.2f}<extra></extra>',
    text=median_by_year['Time'].astype(str),
    visible=True,
    showlegend=True
))

# Add a visual indicator for Denmark's path over time
denmark_data = df_clean[df_clean['IsDenmark']].sort_values('Time')
fig.add_trace(go.Scatter(
    x=denmark_data['GDP_Per_Capita'],
    y=denmark_data['PerCapita'],
    mode='lines',
    line=dict(color='rgba(198, 12, 48, 0.5)', width=2),
    name='Denmark\'s Path',
    hoverinfo='skip',
    visible=True,
    showlegend=True
))

output_filename = '../assets/denmark_tourism_bubble_interactive.html'
fig.write_html(
    output_filename,
    include_plotlyjs='cdn',
    config={'responsive': True, 'displayModeBar': True, 'modeBarButtonsToRemove': ['lasso2d', 'select2d']}
)
print(f"Final animated plot saved: {output_filename}")

Final animated plot saved: ../assets/denmark_tourism_bubble_interactive.html with scrollbar removed
